# Modern Data Analytics [G0Z39a]

## Project: Covid 19 in the USA

### Load packages

In [1]:
import mda_module_008 as mda

import os
import pandas as pd
import numpy as np


import plotly.graph_objs as go
import plotly.io as pio
import plotly.express as px

import plotly.offline as py
py.init_notebook_mode(connected=True)
pio.renderers.default = 'plotly_mimetype'
  
import matplotlib.pyplot as plt

import datetime as dt

import ipywidgets as widgets
from IPython.display import display

from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

### Load data

In [2]:
cwd = os.getcwd()
us = pd.read_csv(cwd+"/covid-19-data/us.csv")
counties_df = pd.read_csv(cwd+"/covid-19-data/us-counties.csv")
states_df = pd.read_csv(cwd+"/covid-19-data/us-states.csv")
counties20 = pd.read_csv(cwd+"/covid-19-data/us-counties-2020.csv")
counties21 = pd.read_csv(cwd+"/covid-19-data/us-counties-2021.csv")
counties22 = pd.read_csv(cwd+"/covid-19-data/us-counties-2022.csv")
extra_data = pd.read_csv(cwd+"/data/extra_data.csv")

### Data Pre-Processing

In [3]:
counties20_clean = mda.counties_preprocessing(counties20)
counties21_clean = mda.counties_preprocessing(counties21)
counties22_clean = mda.counties_preprocessing(counties22)

### Data Info

In [4]:
perstate20 = mda.per_state(counties20_clean)
perstate21 = mda.per_state(counties21_clean)
perstate22 = mda.per_state(counties22_clean)
percounty20 = mda.per_county(counties20_clean)
percounty21 = mda.per_county(counties21_clean)
percounty22 = mda.per_county(counties22_clean)

In [5]:
state_stats20 = pd.DataFrame(counties20_clean.groupby('state').describe())
state_stats21 = pd.DataFrame(counties21_clean.groupby('state').describe())
state_stats22 = pd.DataFrame(counties22_clean.groupby('state').describe())

In [6]:
county_stats20 = pd.DataFrame(counties20_clean.groupby('county').describe())
county_stats21 = pd.DataFrame(counties21_clean.groupby('county').describe())
county_stats22 = pd.DataFrame(counties22_clean.groupby('county').describe())

### Map

In [7]:
mapdf = mda.extra_data_retriever(extra_data, mda.state_per_month(states_df))

In [8]:
def fig_creator(s):
    import plotly.express as px
    
    fig = px.scatter_geo(mapdf, locations="code", locationmode="USA-states", hover_name="state",
                     hover_data=["cases", "deaths", "1_dose", "complete_dose"], size=s, size_max=20,
                     animation_frame="date", projection="albers usa", title="Covid-19 evolution in US per state", width=1000, height=1000)
    fig.show()

widgets.interact(fig_creator,
                 s=widgets.Dropdown(
                     options=[("Cases", "cases"),
                              ("Deaths", "deaths"),
                              ("One Vaccination Dose", "1_dose"),
                              ("Complete Vaccination", "complete_dose")],
                     description='Select:'));

### Time Series plot

In [9]:
us_timeseries = mda.timeseries_process(us, "us")
state_timeseries = mda.timeseries_process(states_df, "state")
counties = pd.concat([counties20_clean, counties21_clean, counties22_clean])
counties_timeseries = mda.timeseries_process(counties, "county")

In [10]:
dropdown_case = widgets.Dropdown(options=[("Daily Cases", 'daily_cases'), ("Daily Deaths", 'daily_deaths')])
dropdown_level = widgets.Dropdown(options=[('USA','us'),('State','state'), ('County','county')])
dropdown_state = widgets.Dropdown(options=state_timeseries['state'].unique())
dropdown_county = widgets.Dropdown(options=counties_timeseries['county'].unique())
input_widgets = widgets.HBox([dropdown_case, dropdown_level])

output = widgets.Output()
def com_filter(case, level, state, county):
    output.clear_output()
    if level == 'us':
        with output:
            mda.plot(us_timeseries, level="us", y=case)
    elif level == "state":
        with output:
            display(dropdown_state)
            mda.plot(state_timeseries, level="state", y=case, state=state)
    elif level == "county":
        with output:
            display(dropdown_county)
            mda.plot(counties_timeseries, level="county", y=case, county=county)

def dropdown_case_eventhandler(change):
    com_filter(change.new, dropdown_level.value, dropdown_state.value, dropdown_county.value)

def dropdown_level_eventhandler(change):
    com_filter(dropdown_case.value, change.new, dropdown_state.value, dropdown_county.value)    
    
def dropdown_state_eventhandler(change):
    com_filter(dropdown_case.value, dropdown_level.value, change.new, dropdown_county.value)
    
def dropdown_county_eventhandler(change):
    com_filter(dropdown_case.value, dropdown_level.value, dropdown_state.value, change.new)
    
dropdown_case.observe(dropdown_case_eventhandler, names='value')
dropdown_level.observe(dropdown_level_eventhandler, names='value')
dropdown_state.observe(dropdown_state_eventhandler, names='value')
dropdown_county.observe(dropdown_county_eventhandler, names='value')
display(input_widgets)


display(output)

### Clustering

In [11]:
df = mapdf.copy(deep=True)

In [12]:
df20_processed, df21_processed, df22_processed = mda.cluster_process(df)

#### 2020

In [13]:
warnings.filterwarnings('ignore')

clusterdf20, Z20, silhoutte20, ari20 = mda.cluster_algorithm(df20_processed, "KMeans")

fig20 = go.Figure(
    data=go.Scatter(
        x=clusterdf20["PC1"].values,
        y=clusterdf20["PC2"].values,
        text=clusterdf20.index,
        mode='markers',
        marker=go.Marker(
            size=df20_processed["deaths"],
            sizemode='diameter',
            sizeref=df20_processed["deaths"].max()/50,
            opacity=1,
            color=Z20
        )
    )
)

fig20.update_layout(
    go.Layout(
        title='US State Cluster Analysis 2020',
        xaxis=go.XAxis(title="PC1", showgrid=True, zeroline=True, showticklabels=True),
        yaxis=go.YAxis(title="PC2", showgrid=True, zeroline=True, showticklabels=True),
        hovermode='closest'
    )
)

fig20.show()

#### 2021

In [14]:
warnings.filterwarnings('ignore')

clusterdf21, Z21, silhoutte21, ari21 = mda.cluster_algorithm(df21_processed, "KMeans")

fig21 = go.Figure(
    data=go.Scatter(
        x=clusterdf21["PC1"].values,
        y=clusterdf21["PC2"].values,
        text=clusterdf21.index,
        mode='markers',
        marker=go.Marker(
            size=df21_processed["deaths"],
            sizemode='diameter',
            sizeref=df21_processed["deaths"].max()/50,
            opacity=1,
            color=Z21
        )
    )
)

fig21.update_layout(
    go.Layout(
        title='US State Cluster Analysis 2021',
        xaxis=go.XAxis(title="PC1", showgrid=True, zeroline=True, showticklabels=True),
        yaxis=go.YAxis(title="PC2", showgrid=True, zeroline=True, showticklabels=True),
        hovermode='closest'
    )
)

fig21.show()

#### 2022

In [15]:
warnings.filterwarnings('ignore')

clusterdf22, Z22, silhoutte22, ari22 = mda.cluster_algorithm(df22_processed, "KMeans")

fig22 = go.Figure(
    data=go.Scatter(
        x=clusterdf22["PC1"].values,
        y=clusterdf22["PC2"].values,
        text=clusterdf22.index,
        mode='markers',
        marker=go.Marker(
            size=df22_processed["deaths"],
            sizemode='diameter',
            sizeref=df22_processed["deaths"].max()/50,
            opacity=1,
            color=Z22
        )
    )
)

fig22.update_layout(
    go.Layout(
        title='US State Cluster Analysis 2022',
        xaxis=go.XAxis(title="PC1", showgrid=True, zeroline=True, showticklabels=True),
        yaxis=go.YAxis(title="PC2", showgrid=True, zeroline=True, showticklabels=True),
        hovermode='closest'
    )
)

fig22.show()

### Linear Regression

In [53]:
def linear_regressor(df, a=1.0):
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import train_test_split
    from sklearn import linear_model
    import pandas as pd
    import numpy as np
    
    new_df = df.drop(["latitude", "longitude", "1_dose", "population", "risk_category"], axis=1)
    
    scaler = StandardScaler()
    scaler.fit_transform(new_df)
    
    X = np.array(new_df.loc[:,["cases", "complete_dose"]].values)
    y = np.array(new_df.loc[:,["deaths"]].values)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size = 0.2, random_state=0)    
    
    ols_model = linear_model.LinearRegression()
    ridge_model = linear_model.Ridge(alpha=a)
    lasso_model = linear_model.Lasso(alpha=a)
    
    b1 = []
    b2 = []
    intercepts = []
    r2_train = []
    r2_test = []
    mses = []
    
    ols_model.fit(X_train, y_train)
    b1.append(ols_model.coef_[0][0])
    b2.append(ols_model.coef_[0][1])
    intercepts.append(ols_model.intercept_[0])
    r2_train.append(ols_model.score(X_train, y_train))
    r2_test.append(ols_model.score(X_test, y_test))
    mses.append(mean_squared_error(y_train, ols_model.predict(X_train))**0.5)
    
    ridge_model.fit(X_train, y_train)
    b1.append(ridge_model.coef_[0][0])
    b2.append(ridge_model.coef_[0][1])
    intercepts.append(ridge_model.intercept_[0])
    r2_train.append(ridge_model.score(X_train, y_train))
    r2_test.append(ridge_model.score(X_test, y_test))
    mses.append(mean_squared_error(y_train, ridge_model.predict(X_train))**0.5)
    
    lasso_model.fit(X_train, y_train)
    b1.append(lasso_model.coef_[0])
    b2.append(lasso_model.coef_[1])
    intercepts.append(lasso_model.intercept_[0])
    r2_train.append(lasso_model.score(X_train, y_train))
    r2_test.append(lasso_model.score(X_test, y_test))
    mses.append(mean_squared_error(y_train, lasso_model.predict(X_train))**0.5)
    
    results = pd.DataFrame({"Model":["OLS", "Ridge", "Lasso"],
                            "a": ["-", a, a],
                            "Intercept": intercepts,
                            "β1": b1,
                            "β2": b2,
                            "R2 training": r2_train,
                            "R2 testing": r2_test,
                            "MSE": mses})
    return results

In [54]:
linear_results = linear_regressor(df21_processed, a=2)

In [55]:
linear_results

,Model,a,Intercept,β1,β2,R2 training,R2 testing,MSE
0,OLS,-,-986.210868,0.016506,-0.000644,0.969022,0.983616,2768.504152
1,Ridge,2,-986.210868,0.016506,-0.000644,0.969022,0.983616,2768.504152
2,Lasso,2,-986.210867,0.016506,-0.000644,0.969022,0.983616,2768.504152


### Linear Mixed Effects Models

In [36]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [37]:
lmem = mda.lmem_process(mapdf)

In [39]:
lmem

,state,month,year,latitude,longitude,cases,deaths,1_dose,complete_dose,population,monthly_cases,monthly_deaths,monthly_1dose,monthly_completedose
0,Alabama,3,2020,32.601011,-86.680736,999,14,0,0,4903185,999,14,0,0
1,Alabama,4,2020,32.601011,-86.680736,7068,272,0,0,4903185,6069,258,0,0
2,Alabama,5,2020,32.601011,-86.680736,17952,630,0,0,4903185,10884,358,0,0
3,Alabama,6,2020,32.601011,-86.680736,38045,950,0,0,4903185,20093,320,0,0
4,Alabama,7,2020,32.601011,-86.680736,87723,1580,0,0,4903185,49678,630,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,Wyoming,25,2022,43.000325,-107.554567,146505,1625,62640,54351,578759,30867,99,3181,3799
241,Wyoming,26,2022,43.000325,-107.554567,155101,1718,63254,55093,578759,8596,93,614,742
242,Wyoming,27,2022,43.000325,-107.554567,156112,1791,63619,55498,578759,1011,73,365,405
243,Wyoming,28,2022,43.000325,-107.554567,156550,1812,64139,56324,578759,438,21,520,826


In [40]:
lmem2020 = lmem[lmem["year"]==2020]
lmem2021 = lmem[lmem["year"]==2021]
lmem2022 = lmem[lmem["year"]==2022]

In [41]:
temp_df = lmem2021.copy(deep=True)
temp_df["month"]-=12
month_dict = {1:"January", 2:"February", 3:"March", 4:"April", 5:"May", 6:"June", 7:"July", 8:"August", 9:"September", 10:"October", 11:"November", 12:"December"}
temp_df["month"] = temp_df["month"].map(month_dict)

def fig_creator2(selection):
    import plotly.express as px
    
    title_dict = {"monthly_cases":"Monthly Cases",
                  "monthly_deaths":"Monthly Deaths",
                  "monthly_1dose":"Monthly Vaccination Dose",
                  "monthly_completedose":"Monthly Complete Vaccinations"}
    
    fig = px.strip(temp_df, x='month', y=selection, color="state", title=f"{title_dict[selection]} per month in 2021",
                   hover_data=["monthly_cases", "monthly_deaths", "monthly_1dose", "monthly_completedose"],
                   labels={selection:title_dict[selection]})
    fig.show()

widgets.interact(fig_creator2,
                 selection=widgets.Dropdown(
                     options=[("Monthly Cases", "monthly_cases"),
                              ("Monthly Deaths", "monthly_deaths"),
                              ("Monthly Vaccination Dose", "monthly_1dose"),
                              ("Monthly Complete Vaccinations", "monthly_completedose")],
                     description='Select:'));

interactive(children=(Dropdown(description='Select:', options=(('Monthly Cases', 'monthly_cases'), ('Monthly D…

In [42]:
temp_df = lmem2021.copy(deep=True)
temp_df["month"]-=12

def fig_creator3(selection1, selection2):
    import plotly.express as px
    
    title_dict = {"monthly_cases":"Monthly Cases",
                  "monthly_deaths":"Monthly Deaths",
                  "monthly_1dose":"Monthly Vaccination Dose",
                  "monthly_completedose":"Monthly Complete Vaccinations"}
    
    fig = px.scatter(temp_df[temp_df["state"]==selection2], x="month", y=selection1, trendline="ols",
                     title=f"{title_dict[selection1]} in {selection2} 2021",
                    labels={selection1:title_dict[selection1]})
    fig.show()

selection2_options = []
for s in lmem2021["state"].unique():
    selection2_options.append((s,s))

widgets.interact(fig_creator3,
                 
                 selection1=widgets.Dropdown(
                     options=[("Monthly Cases", "monthly_cases"),
                              ("Monthly Deaths", "monthly_deaths"),
                              ("Monthly Vaccination Dose", "monthly_1dose"),
                              ("Monthly Complete Vaccinations", "monthly_completedose")],
                     description='Select:'),
                 
                 selection2=widgets.Dropdown(
                     options=selection2_options,
                     description='Select:'));

interactive(children=(Dropdown(description='Select:', options=(('Monthly Cases', 'monthly_cases'), ('Monthly D…

In [43]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [59]:
temp_df = lmem2021.copy(deep=True)
temp_df = temp_df[["state", "month", "cases", "deaths"]]
temp_df["month"]-=12
month_dict = {1:"January", 2:"February", 3:"March", 4:"April", 5:"May", 6:"June", 7:"July", 8:"August", 9:"September", 10:"October", 11:"November", 12:"December"}
temp_df["month"] = temp_df["month"].map(month_dict)

In [60]:
# Run LMER
md = smf.mixedlm("cases ~ month", temp_df, groups=temp_df["state"], re_formula="~month")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

                                  Mixed Linear Model Regression Results
Model:                           MixedLM                Dependent Variable:                cases          
No. Observations:                588                    Method:                            REML           
No. Groups:                      49                     Scale:                             1365545318.1121
Min. group size:                 12                     Log-Likelihood:                    -7292.9412     
Max. group size:                 12                     Converged:                         Yes            
Mean group size:                 12.0                                                                     
----------------------------------------------------------------------------------------------------------
                                                Coef.        Std.Err.    z    P>|z|    [0.025     0.975]  
------------------------------------------------------------------------

C:\Users\35799\anaconda3\envs\mda_2022\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2261: ConvergenceWarning:

The Hessian matrix at the estimated parameter values is not positive definite.



In [61]:
# Run LMER
md = smf.mixedlm("deaths ~ month", temp_df, groups=temp_df["state"], re_formula="~month")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

                              Mixed Linear Model Regression Results
Model:                          MixedLM              Dependent Variable:              deaths     
No. Observations:               588                  Method:                          REML       
No. Groups:                     49                   Scale:                           422353.3157
Min. group size:                12                   Log-Likelihood:                  -5005.4899 
Max. group size:                12                   Converged:                       Yes        
Mean group size:                12.0                                                             
-------------------------------------------------------------------------------------------------
                                              Coef.     Std.Err.   z    P>|z|   [0.025    0.975] 
-------------------------------------------------------------------------------------------------
Intercept                                     1166

C:\Users\35799\anaconda3\envs\mda_2022\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2261: ConvergenceWarning:

The Hessian matrix at the estimated parameter values is not positive definite.



In [62]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
 
tempdf_scaled = scaler.fit_transform(temp_df.drop(["state", "month"], axis=1).to_numpy())
tempdf_scaled = pd.DataFrame(tempdf_scaled, columns=['cases', 'deaths'])
tempdf_scaled["state"]=temp_df["state"]
tempdf_scaled["month"]=temp_df["month"]

In [64]:
# Run LMER
md = smf.mixedlm("cases ~ month", tempdf_scaled, groups=tempdf_scaled["state"], re_formula="~month")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

C:\Users\35799\anaconda3\envs\mda_2022\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning:

The MLE may be on the boundary of the parameter space.



                        Mixed Linear Model Regression Results
Model:                       MixedLM           Dependent Variable:           cases   
No. Observations:            588               Method:                       REML    
No. Groups:                  49                Scale:                        0.0018  
Min. group size:             12                Log-Likelihood:               582.6030
Max. group size:             12                Converged:                    Yes     
Mean group size:             12.0                                                    
-------------------------------------------------------------------------------------
                                           Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------
Intercept                                  -0.123    0.050 -2.429 0.015 -0.222 -0.024
month[T.August]                             0.162    0.016  9.850 0.000  0.130

C:\Users\35799\anaconda3\envs\mda_2022\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2261: ConvergenceWarning:

The Hessian matrix at the estimated parameter values is not positive definite.



In [65]:
# Run LMER
md = smf.mixedlm("deaths ~ month", tempdf_scaled, groups=tempdf_scaled["state"], re_formula="~month")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

C:\Users\35799\anaconda3\envs\mda_2022\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning:

The MLE may be on the boundary of the parameter space.



                        Mixed Linear Model Regression Results
Model:                       MixedLM           Dependent Variable:           deaths  
No. Observations:            588               Method:                       REML    
No. Groups:                  49                Scale:                        0.0019  
Min. group size:             12                Log-Likelihood:               526.8008
Max. group size:             12                Converged:                    Yes     
Mean group size:             12.0                                                    
-------------------------------------------------------------------------------------
                                           Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------
Intercept                                  -0.077    0.061 -1.260 0.208 -0.196  0.043
month[T.August]                             0.088    0.013  6.854 0.000  0.063

C:\Users\35799\anaconda3\envs\mda_2022\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2261: ConvergenceWarning:

The Hessian matrix at the estimated parameter values is not positive definite.

